In [ ]:
!pip install xgboost

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

# from cats2d.rd_cats2d import CATS2D
# from rdkit.Avalon import pyAvalonTools
# from rdkit import Chem
# from rdkit.Chem import Crippen,Descriptors, MACCSkeys, GraphDescriptors, Lipinski, QED
from statistics import mean
from tqdm import tqdm

In [2]:
from xgboost import XGBRegressor, XGBClassifier
import sklearn
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error, roc_auc_score, f1_score

In [3]:
f = open('Balanced_dataset.txt')
lines = f.readlines()
class_data = open("Class_data.txt", "w+")
class_data.write('class;Sequence'+'\n')

i = 0
for line in lines:
  if i % 2:
    class_data.write(line + '\n')
  else:
    class_data.write(line[-2] + ';')
  i += 1

class_data.close()


In [4]:
df = pd.read_csv("Class_data.txt", sep=';')
df

,class,Sequence
0,0,RRRRRRRGGIYLATALAKWALKQGF
1,0,IYLATALAKWALKQGFGGRRRRRRR
2,0,RRRRRRRGGKLAKLAKKLAKLAK
3,0,KKALLAHALHLLALLALHLAHALKKA
4,0,GALFLGFLGAAGSTMGAWSQPKSKRKV
...,...,...
919,1,QLEINNIPCKICIIVIGPCWEERTGTTQERPSEAKHDIWYKTNQQGARH
920,1,MFPWDQNCSCFRKRHVFMCKEYRYQVDPRCMHDIYHFSSFWDSMDMVLV
921,1,RCYYNAGSVDACCLKCWQWGASCWNQLRLPCKGVSPNCCNSTARER...
922,1,HRYCVCFYFYAFMGTMTKFLMVAPTFEFCLNLAIFPNLFMHMSVDC...


In [9]:
df['class'].value_counts()

class
0    462
1    462
Name: count, dtype: int64

##CTD

In [ ]:
!pip install pybiomed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybiomed: filename=PyBioMed-1.0-py3-none-any.whl size=11266895 sha256=d1e842b64f41af65b13ee1c15aa226c74c1abd9be8179fed67a137991f26f0e1
  Stored in directory: /root/.cache/pip/wheels/03/c8/be/74f7e3ad04bbbb4a2977551bc19e784919a025b97ba6da1a17
Successfully built pybiomed


In [ ]:
from PyBioMed.PyProtein.CTD import (
    CalculateCTD)

In [ ]:
data = CalculateCTD(df['Sequence'][0])
new_data = {k: list([v]) for k, v in data.items()}
res = pd.DataFrame.from_dict(new_data)
res

,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,_ChargeC1,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0.32,0.2,0.48,0.52,0.4,0.08,0.68,0.2,0.12,0.36,...,32.0,36.0,52.0,60.0,96.0,40.0,40.0,64.0,76.0,100.0


In [ ]:
for i in range(1, df.shape[0]):
  slov = CalculateCTD(df['Sequence'][i])
  res = pd.concat([res, pd.DataFrame.from_records([slov])] ,ignore_index=True)
res

,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,_ChargeC1,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,0.120,0.360,...,32.000,36.000,52.000,60.000,96.000,40.000,40.000,64.000,76.000,100.000
1,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,0.120,0.360,...,8.000,16.000,24.000,44.000,72.000,4.000,4.000,28.000,40.000,64.000
2,0.261,0.174,0.565,0.435,0.565,0.000,0.913,0.000,0.087,0.565,...,34.783,34.783,52.174,65.217,95.652,47.826,47.826,60.870,78.261,91.304
3,0.308,0.385,0.308,0.692,0.154,0.154,1.000,0.000,0.000,0.154,...,11.538,26.923,50.000,76.923,100.000,15.385,19.231,46.154,57.692,88.462
4,0.481,0.222,0.296,0.593,0.185,0.222,0.481,0.185,0.333,0.148,...,3.704,22.222,44.444,59.259,85.185,11.111,14.815,25.926,55.556,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,0.224,0.551,0.224,0.388,0.388,0.224,0.408,0.367,0.224,0.122,...,16.327,48.980,63.265,81.633,100.000,4.082,14.286,26.531,32.653,79.592
920,0.184,0.327,0.490,0.347,0.306,0.347,0.367,0.388,0.245,0.122,...,6.122,30.612,57.143,73.469,87.755,2.041,20.408,53.061,81.633,100.000
921,0.288,0.462,0.250,0.462,0.308,0.231,0.365,0.327,0.308,0.135,...,5.769,13.462,42.308,69.231,98.077,3.846,25.000,44.231,59.615,100.000
922,0.214,0.411,0.375,0.536,0.161,0.304,0.375,0.464,0.161,0.054,...,1.786,17.857,41.071,64.286,100.000,7.143,23.214,50.000,69.643,98.214


In [ ]:
df = pd.concat([df, res], axis=1, join='inner')

In [ ]:
df

,class,Sequence,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0,RRRRRRRGGIYLATALAKWALKQGF,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,...,32.000,36.000,52.000,60.000,96.000,40.000,40.000,64.000,76.000,100.000
1,0,IYLATALAKWALKQGFGGRRRRRRR,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,...,8.000,16.000,24.000,44.000,72.000,4.000,4.000,28.000,40.000,64.000
2,0,RRRRRRRGGKLAKLAKKLAKLAK,0.261,0.174,0.565,0.435,0.565,0.000,0.913,0.000,...,34.783,34.783,52.174,65.217,95.652,47.826,47.826,60.870,78.261,91.304
3,0,KKALLAHALHLLALLALHLAHALKKA,0.308,0.385,0.308,0.692,0.154,0.154,1.000,0.000,...,11.538,26.923,50.000,76.923,100.000,15.385,19.231,46.154,57.692,88.462
4,0,GALFLGFLGAAGSTMGAWSQPKSKRKV,0.481,0.222,0.296,0.593,0.185,0.222,0.481,0.185,...,3.704,22.222,44.444,59.259,85.185,11.111,14.815,25.926,55.556,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,1,QLEINNIPCKICIIVIGPCWEERTGTTQERPSEAKHDIWYKTNQQGARH,0.224,0.551,0.224,0.388,0.388,0.224,0.408,0.367,...,16.327,48.980,63.265,81.633,100.000,4.082,14.286,26.531,32.653,79.592
920,1,MFPWDQNCSCFRKRHVFMCKEYRYQVDPRCMHDIYHFSSFWDSMDMVLV,0.184,0.327,0.490,0.347,0.306,0.347,0.367,0.388,...,6.122,30.612,57.143,73.469,87.755,2.041,20.408,53.061,81.633,100.000
921,1,RCYYNAGSVDACCLKCWQWGASCWNQLRLPCKGVSPNCCNSTARER...,0.288,0.462,0.250,0.462,0.308,0.231,0.365,0.327,...,5.769,13.462,42.308,69.231,98.077,3.846,25.000,44.231,59.615,100.000
922,1,HRYCVCFYFYAFMGTMTKFLMVAPTFEFCLNLAIFPNLFMHMSVDC...,0.214,0.411,0.375,0.536,0.161,0.304,0.375,0.464,...,1.786,17.857,41.071,64.286,100.000,7.143,23.214,50.000,69.643,98.214


# AAComposition

In [ ]:
from PyBioMed.PyProtein.AAComposition import (
    CalculateAAComposition,
    CalculateDipeptideComposition
)

In [ ]:
data = CalculateAAComposition(df['Sequence'][0])
new_data = {k: list([v]) for k, v in data.items()}
res = pd.DataFrame.from_dict(new_data)
for i in range(1, df.shape[0]):
  slov = CalculateAAComposition(df['Sequence'][i])
  #print(slov)
  #new_slov = {k: list([v]) for k, v in slov.items()}
  res = pd.concat([res, pd.DataFrame.from_records([slov])] ,ignore_index=True)
df = pd.concat([df, res], axis=1, join='inner')

In [ ]:
data = CalculateDipeptideComposition(df['Sequence'][0])
new_data = {k: list([v]) for k, v in data.items()}
res = pd.DataFrame.from_dict(new_data)
res

,AA,AR,AN,AD,AC,AE,AQ,AG,AH,AI,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in range(1, df.shape[0]):
  slov = CalculateDipeptideComposition(df['Sequence'][i])
  #print(slov)
  #new_slov = {k: list([v]) for k, v in slov.items()}
  res = pd.concat([res, pd.DataFrame.from_records([slov])] ,ignore_index=True)
res

,AA,AR,AN,AD,AC,AE,AQ,AG,AH,AI,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,8.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,3.85,0.00,0.0,0.0,0.00,0.0,0.0,3.85,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,0.00,2.08,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
920,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,2.08,0.0,0.0,2.08,0.0,0.00,0.0,0.0,0.0,0.0
921,0.00,1.96,0.0,0.0,1.96,0.0,0.0,1.96,0.0,0.00,...,0.00,0.0,0.0,0.00,0.0,1.96,0.0,0.0,0.0,0.0
922,0.00,1.82,0.0,0.0,0.00,0.0,0.0,0.00,0.0,1.82,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [ ]:
df = pd.concat([df, res], axis=1, join='inner')
df

,class,Sequence,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,0,RRRRRRRGGIYLATALAKWALKQGF,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,0,IYLATALAKWALKQGFGGRRRRRRR,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,0,RRRRRRRGGKLAKLAKKLAKLAK,0.261,0.174,0.565,0.435,0.565,0.000,0.913,0.000,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,0,KKALLAHALHLLALLALHLAHALKKA,0.308,0.385,0.308,0.692,0.154,0.154,1.000,0.000,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,0,GALFLGFLGAAGSTMGAWSQPKSKRKV,0.481,0.222,0.296,0.593,0.185,0.222,0.481,0.185,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,1,QLEINNIPCKICIIVIGPCWEERTGTTQERPSEAKHDIWYKTNQQGARH,0.224,0.551,0.224,0.388,0.388,0.224,0.408,0.367,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
920,1,MFPWDQNCSCFRKRHVFMCKEYRYQVDPRCMHDIYHFSSFWDSMDMVLV,0.184,0.327,0.490,0.347,0.306,0.347,0.367,0.388,...,2.08,0.0,0.0,2.08,0.0,0.00,0.0,0.0,0.0,0.0
921,1,RCYYNAGSVDACCLKCWQWGASCWNQLRLPCKGVSPNCCNSTARER...,0.288,0.462,0.250,0.462,0.308,0.231,0.365,0.327,...,0.00,0.0,0.0,0.00,0.0,1.96,0.0,0.0,0.0,0.0
922,1,HRYCVCFYFYAFMGTMTKFLMVAPTFEFCLNLAIFPNLFMHMSVDC...,0.214,0.411,0.375,0.536,0.161,0.304,0.375,0.464,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [ ]:
df.drop('Sequence', axis=1, inplace=True)
df

,class,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,...,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
0,0,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,0.120,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,0,0.320,0.200,0.480,0.520,0.400,0.080,0.680,0.200,0.120,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,0,0.261,0.174,0.565,0.435,0.565,0.000,0.913,0.000,0.087,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,0,0.308,0.385,0.308,0.692,0.154,0.154,1.000,0.000,0.000,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,0,0.481,0.222,0.296,0.593,0.185,0.222,0.481,0.185,0.333,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,1,0.224,0.551,0.224,0.388,0.388,0.224,0.408,0.367,0.224,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
920,1,0.184,0.327,0.490,0.347,0.306,0.347,0.367,0.388,0.245,...,2.08,0.0,0.0,2.08,0.0,0.00,0.0,0.0,0.0,0.0
921,1,0.288,0.462,0.250,0.462,0.308,0.231,0.365,0.327,0.308,...,0.00,0.0,0.0,0.00,0.0,1.96,0.0,0.0,0.0,0.0
922,1,0.214,0.411,0.375,0.536,0.161,0.304,0.375,0.464,0.161,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


# Classificator

In [ ]:
features_list = df.iloc[:, 1:].columns
def select_features(X, y):
    fs = SelectKBest(score_func=mutual_info_regression, k=395)
    fs.fit(X, y)
    clas_labels = features_list[fs.get_support()]
    X_fs = fs.transform(X)
    return X_fs, fs, clas_labels

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
X_fs, fs, clas_labels = select_features(X, y)

In [ ]:
cols_idxs = fs.get_support(indices=True)
df = df.iloc[:, cols_idxs]

In [ ]:
split = KFold(n_splits=5, random_state=41, shuffle=True)
scores= {
         "F1":  make_scorer(f1_score),
         "AUC": make_scorer(roc_auc_score, needs_threshold=True)
         }

In [ ]:
X_train, X_test, y_train, y_test = X, X, y, y

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
!nvidia-smi

Sun Jul 21 22:08:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# GridSearch

In [ ]:
parameters = {
    'max_depth': range(6, 12, 2),
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [1, 0.75, 0.5, 0.3, 0.1],
    'n_estimators': [1000, 1500, 2000],
}

In [ ]:
roc_auc_scorer = sklearn.metrics.make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)

In [ ]:
grid_search = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=parameters,
    # scoring='neg_mean_squared_error',
    scoring=roc_auc_scorer,
    verbose=3)

In [ ]:
grid_search.fit(X, y)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=1;, score=0.960 total time=   6.0s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=1;, score=0.950 total time=   5.3s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=1;, score=0.972 total time=   3.2s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=1;, score=0.964 total time=   3.1s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=1;, score=0.830 total time=   2.8s
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=0.75;, score=0.959 total time=   4.5s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=0.75;, score=0.954 total time=   3.1s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=1000, subsample=0.75;, score=0.971 total time=   3.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimator

KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

# XGB

In [ ]:
model = XGBClassifier(random_state=42, learning_rate=0.01, max_depth=8, n_estimators=1500, subsample=0.3, tree_method='gpu_hist', gpu_id=0)

In [ ]:
cv_scores = cross_validate(model, X_train, y_train, scoring=scores, cv=split)
print(f"On cross-validation:")
print(f"Mean ROC_AUC score is {cv_scores['test_AUC'].mean().round(3)} ± {cv_scores['test_AUC'].std().round(3)}")
print(f"Mean F1 score is {cv_scores['test_F1'].mean().round(3)} ± {cv_scores['test_F1'].std().round(3)}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:08:36] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:08:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:08:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [

On cross-validation:
Mean ROC_AUC score is 0.976 ± 0.006
Mean F1 score is 0.909 ± 0.025


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:09:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"F1: {f1_score(y_test, y_pred).round(3)}")

y_pred = model.predict_proba(X_test)
print(f"ROC_AUC: {roc_auc_score(y_test, y_pred[:, 1]).round(3)}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:09:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


F1: 0.997
ROC_AUC: 1.0


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:09:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


# Class label prediction

In [ ]:
df = pd.read_csv('without_NaN.csv')
df

,Unnamed: 0,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target,_PolarizabilityC1,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0,Mean Fluorescence intensity,23.0,5,8,RKKRRQRRR,0.250000,12.50,650.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
1,1,Mean Fluorescence intensity,23.0,5,8,RKKRRQRR,0.166667,12.50,31.25,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
2,2,Mean Fluorescence intensity,23.0,5,8,RKKRRQR,0.166667,12.50,10.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
3,3,Mean Fluorescence intensity,23.0,5,8,KKRRQRRR,0.166667,12.50,22.50,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
4,4,Mean Fluorescence intensity,23.0,5,8,KRRQRRR,0.166667,12.50,20.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,1437,Cellular uptake,37.0,5,30,CGRKKRRQR-Ahx-RRPPQ,0.500000,2.10,0.85,0.111,...,33.333,33.333,33.333,33.333,33.333,0.0,0.0,0.0,0.0,0.0
1438,1438,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.90,0.111,...,44.444,44.444,44.444,44.444,44.444,0.0,0.0,0.0,0.0,0.0
1439,1439,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.74,0.111,...,55.556,55.556,55.556,55.556,55.556,0.0,0.0,0.0,0.0,0.0
1440,1440,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.78,0.111,...,66.667,66.667,66.667,66.667,66.667,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_to_class_label_pred = df.copy()

In [ ]:
data = CalculateAAComposition(df['Sequence'][0])
new_data = {k: list([v]) for k, v in data.items()}
res = pd.DataFrame.from_dict(new_data)
for i in range(1, df_to_class_label_pred.shape[0]):
  slov = CalculateAAComposition(df_to_class_label_pred['Sequence'][i])
  #print(slov)
  #new_slov = {k: list([v]) for k, v in slov.items()}
  res = pd.concat([res, pd.DataFrame.from_records([slov])] ,ignore_index=True)
df_to_class_label_pred = pd.concat([df_to_class_label_pred, res], axis=1, join='inner')

In [ ]:
data = CalculateDipeptideComposition(df['Sequence'][0])
new_data = {k: list([v]) for k, v in data.items()}
res = pd.DataFrame.from_dict(new_data)
for i in range(1, df_to_class_label_pred.shape[0]):
  slov = CalculateDipeptideComposition(df_to_class_label_pred['Sequence'][i])
  #print(slov)
  #new_slov = {k: list([v]) for k, v in slov.items()}
  res = pd.concat([res, pd.DataFrame.from_records([slov])] ,ignore_index=True)
df_to_class_label_pred = pd.concat([df_to_class_label_pred, res], axis=1, join='inner')
df

,Unnamed: 0,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target,_PolarizabilityC1,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0,Mean Fluorescence intensity,23.0,5,8,RKKRRQRRR,0.250000,12.50,650.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
1,1,Mean Fluorescence intensity,23.0,5,8,RKKRRQRR,0.166667,12.50,31.25,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
2,2,Mean Fluorescence intensity,23.0,5,8,RKKRRQR,0.166667,12.50,10.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
3,3,Mean Fluorescence intensity,23.0,5,8,KKRRQRRR,0.166667,12.50,22.50,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
4,4,Mean Fluorescence intensity,23.0,5,8,KRRQRRR,0.166667,12.50,20.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,1437,Cellular uptake,37.0,5,30,CGRKKRRQR-Ahx-RRPPQ,0.500000,2.10,0.85,0.111,...,33.333,33.333,33.333,33.333,33.333,0.0,0.0,0.0,0.0,0.0
1438,1438,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.90,0.111,...,44.444,44.444,44.444,44.444,44.444,0.0,0.0,0.0,0.0,0.0
1439,1439,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.74,0.111,...,55.556,55.556,55.556,55.556,55.556,0.0,0.0,0.0,0.0,0.0
1440,1440,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.78,0.111,...,66.667,66.667,66.667,66.667,66.667,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df_to_class_label_pred = df_to_class_label_pred[clas_labels]
# df_to_class_label_pred

,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,_ChargeC1,...,VH,VL,VM,VF,VP,VS,VT,VW,VY,VV
0,0.000,0.111,0.889,0.000,1.000,0.0,1.0,0.0,0.0,0.889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000,0.125,0.875,0.000,1.000,0.0,1.0,0.0,0.0,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.143,0.857,0.000,1.000,0.0,1.0,0.0,0.0,0.857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.125,0.875,0.000,1.000,0.0,1.0,0.0,0.0,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.143,0.857,0.000,1.000,0.0,1.0,0.0,0.0,0.857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,0.111,0.111,0.778,0.111,0.889,0.0,1.0,0.0,0.0,0.778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438,0.111,0.111,0.778,0.111,0.889,0.0,1.0,0.0,0.0,0.778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1439,0.111,0.111,0.778,0.111,0.889,0.0,1.0,0.0,0.0,0.778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1440,0.111,0.000,0.889,0.111,0.889,0.0,1.0,0.0,0.0,0.889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df["Class_label"] = np.nan

In [ ]:
class_labels = model.predict(df_to_class_label_pred.iloc[:, 9:].values)
for i in range(1442):
  df['Class_label'][i] = class_labels[i]
df

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-55-12f695e81c0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class_label'][i] = class_labels[i]
<ipython-input-55-12f695e81c0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class_label'][i] = class_labels[i]
<ipython-input-55-12f695e81c0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class_label'][i] = class_labels[i]
<ipython-in

,Unnamed: 0,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target,_PolarizabilityC1,...,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100,Class_label
0,0,Mean Fluorescence intensity,23.0,5,8,RKKRRQRRR,0.250000,12.50,650.00,0.000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Mean Fluorescence intensity,23.0,5,8,RKKRRQRR,0.166667,12.50,31.25,0.000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Mean Fluorescence intensity,23.0,5,8,RKKRRQR,0.166667,12.50,10.00,0.000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Mean Fluorescence intensity,23.0,5,8,KKRRQRRR,0.166667,12.50,22.50,0.000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Mean Fluorescence intensity,23.0,5,8,KRRQRRR,0.166667,12.50,20.00,0.000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,1437,Cellular uptake,37.0,5,30,CGRKKRRQR-Ahx-RRPPQ,0.500000,2.10,0.85,0.111,...,33.333,33.333,33.333,33.333,0.0,0.0,0.0,0.0,0.0,0.0
1438,1438,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.90,0.111,...,44.444,44.444,44.444,44.444,0.0,0.0,0.0,0.0,0.0,0.0
1439,1439,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.74,0.111,...,55.556,55.556,55.556,55.556,0.0,0.0,0.0,0.0,0.0,0.0
1440,1440,Cellular uptake,37.0,5,30,CGRKKR-Ahx-RQR-Ahx-R-ahx-RPPQ,0.500000,2.10,0.78,0.111,...,66.667,66.667,66.667,66.667,0.0,0.0,0.0,0.0,0.0,0.0
